# NKSOPS_03_DataTransformation_01

In [ ]:
#load Enviornment
%run NKSOPS_02_ClassAndMethods_01.ipynb

In [ ]:
# Initalize
NKS_OPS_OUTLOOK_EXPORTS_INFO = NKS_OPS(input_path = r"C:\Users\76350\Documents\NKS_OPS\03_Data\01_Raw_Data\DataFiles\MXEmail\*.csv"
                                   , export_parent_dir = r"C:\Users\76350\Documents\NKS_OPS\03_Data\02_Processed_Data"
                                   , input_columns = None
                                   , export_columns = ['Body',"Unique_ID","From: (Name)","FILE_PATH"]
                                   , research_topic = 'NKS_OPS_Data_Study'
                                   , severity = 'H'
                                   , logfile = False
                                   , sqlite_create = r"C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataBase\NKS_OPS_DB_sqlite.db")

In [ ]:
#1 Execute method to get impoted email files, add unique_id/ filepath, and
NKS_OPS_OUTLOOK_EXPORTS_INFO.NKS_OPS_Data_Study()

#1.a Append to single dataframe and rename column
Appened_DF = NKS_OPS_OUTLOOK_EXPORTS_INFO.Append_to_DF()
Appened_DF.rename(columns={"From: (Name)": 'From_Name'}, inplace=True)

In [ ]:
#2 Parse and Extract Sections of Body and create New columns

#2.a MESSAGE_TYPE
NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=Appened_DF, newcolumn='MESSAGETYPE', textcolumn='Body', start='MX', end='MESSAGE')
Appened_DF.loc[:,'MESSAGETYPE'] = 'MX' + Appened_DF.loc[:,'MESSAGETYPE'] + 'MESSAGE'
grouped_messages = Appened_DF.loc[:,'MESSAGETYPE'].dropna().unique()

new_df = pd.DataFrame()
for message in grouped_messages:
    data_item = Appened_DF.copy()
    data_item = data_item[data_item.loc[:,'MESSAGETYPE'].str.contains(message)]
    
    if message == "MX AIRCRAFT OOS MESSAGE":
        #2.b Top Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DATE', textcolumn='Body', start='DATE: ', end='TAIL: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TAIL', textcolumn='Body', start='TAIL: ', end='STATION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STATION', textcolumn='Body', start='STATION: ', end='DESCRIPTION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESCRIPTION', textcolumn='Body', start='DESCRIPTION: ', end=r"STD..GMT...")
        data_item.loc[:,'TROUBLESHOOTING'] = ''
        data_item.loc[:,'CORRECTIVEACTION'] = ''

        #2.c Bottom Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STD_GMT', textcolumn='Body', start= r"STD..GMT...", end=r"ETRS..LT...")
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='ETRS_DATE', textcolumn='Body', start=r"ETRS..LT...", end='DESTINATION: ')
        data_item.loc[:,'RTS_DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESTINATION', textcolumn='Body', start="DESTINATION: ", end="\r\n")
           
        #2.d Make columns visible in csv
        data_item.loc[:,'TAIL'] = data_item.loc[:,'TAIL'].str[:3]
        data_item.loc[:,'STATION'] = data_item.loc[:,'STATION'].str[:3]
        new_df= new_df.append(data_item,ignore_index=True)
        
    if message == "MX AIRCRAFT OOS UPDATE MESSAGE":

        #2.e Top Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DATE', textcolumn='Body', start='DATE: ', end='TAIL: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TAIL', textcolumn='Body', start='TAIL: ', end='STATION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STATION', textcolumn='Body', start='STATION: ', end='DESCRIPTION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESCRIPTION', textcolumn='Body', start='DESCRIPTION: ', end='TROUBLESHOOTING: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TROUBLESHOOTING', textcolumn='Body', start='TROUBLESHOOTING: ', end=r"STD..GMT...")
        data_item.loc[:,'CORRECTIVEACTION'] = ''

        #2.f Bottom Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STD_GMT', textcolumn='Body', start= r"STD..GMT...", end=r"ETRS..LT...")
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='ETRS_DATE', textcolumn='Body', start=r"ETRS..LT...", end='DESTINATION: ')
        data_item.loc[:,'RTS_DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESTINATION', textcolumn='Body', start="DESTINATION: ", end="\r\n")
        
        #2.g Make columns visible in csv
        data_item.loc[:,'TAIL'] = data_item.loc[:,'TAIL'].str[:3]
        data_item.loc[:,'STATION'] = data_item.loc[:,'STATION'].str[:3]
        new_df= new_df.append(data_item,ignore_index=True)
              
    if message == "MX AIRCRAFT OOS RETURN TO SERVICE MESSAGE":

        #2.h Top Section of Message
        data_item.loc[:,'DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TAIL', textcolumn='Body', start='TAIL: ', end='STATION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STATION', textcolumn='Body', start='STATION: ', end='DESCRIPTION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESCRIPTION', textcolumn='Body', start='DESCRIPTION: ', end='CORRECTIVE ACTION: ')
        data_item.loc[:,'TROUBLESHOOTING'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='CORRECTIVEACTION', textcolumn='Body', start='CORRECTIVE ACTION: ', end=r"STD..GMT...")

        #2.i Bottom Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STD_GMT', textcolumn='Body', start=r"STD..GMT...", end='RTS DATE: ')
        data_item.loc[:,'ETRS_DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='RTS_DATE', textcolumn='Body', start='RTS DATE: ', end='DESTINATION: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESTINATION', textcolumn='Body', start='DESTINATION: ', end="\r\n")
        
        #2.j Make columns visible in csv
        data_item.loc[:,'TAIL'] = data_item.loc[:,'TAIL'].str[:3]
        data_item.loc[:,'STATION'] = data_item.loc[:,'STATION'].str[:3]
        new_df= new_df.append(data_item,ignore_index=True)




In [ ]:
#2.k Export results to cs
new_df['Unique_ID'] = new_df['Unique_ID'].astype(str) + "_" + new_df['ID'].astype(str)+ "_" + new_df['MESSAGETYPE'].astype(str) + "_" + new_df['TAIL'].astype(str) + "_" + new_df['STATION'].astype(str) + "_" + new_df['DESTINATION'].astype(str)
new_df['Unique_ID'] = new_df['Unique_ID'].str.replace(' ','')
new_df['DATE'] = new_df['DATE'].str.replace('\r\n','')
new_df['STD_GMT'] = new_df['STD_GMT'].str.replace('\r\n','')
new_df['ETRS_DATE'] = new_df['ETRS_DATE'].str.replace('\r\n','')
new_df['RTS_DATE'] = new_df['RTS_DATE'].str.replace('\r\n','')
new_df[['DATE_DATE','DATE_TIME']] = new_df['DATE'].str.split(' ',expand=True)
new_df[['DATE_DATE','DATE_TIME']] = new_df['DATE'].str.split(' ',expand=True)
new_df.to_csv(r'C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataFiles\Finial.csv', index=False)
new_df

In [ ]:
# #export to sqlite3
# NKS_OPS_OUTLOOK_EXPORTS_INFO.NKS_OPS_DB(dbname=NKS_OPS_OUTLOOK_EXPORTS_INFO.sqlite_create)
# num_rows = len(new_df)
# conn = NKS_OPS_OUTLOOK_EXPORTS_INFO.create_connection(db_file = NKS_OPS_OUTLOOK_EXPORTS_INFO.sqlite_create)
# cur = conn.cursor()
# for opsrow in range(num_rows):
#     try:
#         new_df.iloc[opsrow:opsrow+1].to_sql(name='NKS_OPS', con=conn, if_exists='append', chunksize=1000, index=False)
#     except exc.IntegrityError:
#         #Ignore duplicates
#         pass